**Segundo Trabalho da disciplina Processamento de Dados em Larga Escala**

**Discentes:**

*   **Valéria Cristina A. R. de Figueredo -- vcarf@cesar.school**
*   **Manuela de Lacerda Bezerra Carvalho -- mlbc@cesar.school**


**Docente: Anderson Neves**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pyspark

## Sobre os dados

O arquivo CSV contém eventos 'click' ou 'view' no tempo, de usuários em anúncios de determinadas campanhas.

**Descrição das colunas:**  
timestamp,user_id,action,adId,campaignId

**Amostra:**  
2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01  
2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02  
2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02  
2016-03-26 06:27:00,78aa2467-b502-413b-94e9-04ec8210bd13,click,adId_07,campaignId_03

**Nome do arquivo CSV:**  
data/ad_action.csv

## Sobre as questões

As questões devem ser respondidas usando alguma API da tecnologia Spark, exceto a API "Pandas API on Spark".

Quando utilizar uma action do Spark tenha cuidado para evitar estouro de memória, sempre imaginado que vai executar o código com uma grande massa de dados.

Mesmo que não consiga terminar alguma questão, favor enviar, porque parte do código pode valer alguma pontuação.

In [ ]:
import os
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql import SparkSession

os.environ['PYSPARK_SUBMIT_ARGS'] = '\
    --driver-memory 2G \
    --executor-memory 2G \
    pyspark-shell'

In [ ]:
spark = SparkSession.builder\
    .master("local[*]")\
    .getOrCreate()
data_spark = spark.read.csv('drive/MyDrive/data/ad_action.csv', header=False, inferSchema=True)\
    .toDF('timestamp', 'user_id', 'action', 'adId', 'campaignId')
data_spark.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- action: string (nullable = true)
 |-- adId: string (nullable = true)
 |-- campaignId: string (nullable = true)



In [ ]:
data_spark.show(5)

+-------------------+--------------------+------+-------+-------------+
|          timestamp|             user_id|action|   adId|   campaignId|
+-------------------+--------------------+------+-------+-------------+
|2016-09-21 22:11:00|7c74953c-66cc-48b...| click|adId_09|campaignId_01|
|2016-06-25 18:29:00|676a083e-2f8e-4ff...|  view|adId_09|campaignId_02|
|2016-02-14 19:03:00|77158997-0dfa-48b...| click|adId_02|campaignId_02|
|2016-03-26 06:27:00|78aa2467-b502-413...| click|adId_07|campaignId_03|
|2016-01-02 04:57:00|fef9a98c-d73e-48e...|  view|adId_02|campaignId_02|
+-------------------+--------------------+------+-------+-------------+
only showing top 5 rows



In [ ]:
# Descomente para desligar clusters

# spark.stop()

## 1) Quais são as top 3 campanhas que geraram mais eventos? Ordene pela quantidade de eventos (2,5 pontos)

In [ ]:
data_spark.groupby('campaignId')\
  .count()\
  .orderBy(F.desc('count'))\
  .take(3)

[Row(campaignId='campaignId_02', count=91216),
 Row(campaignId='campaignId_03', count=87036),
 Row(campaignId='campaignId_01', count=76461)]

**Resposta:**

As top 3 campanhas que geraram mais eventos foram as campanhas de ID 02, 03 e 01, como exposto acima.



## 2) Qual campanha teve mais clicks? (2,5 pontos)

In [ ]:
# Contar os cliques por campanha
df_clicks = data_spark.filter(data_spark['action'] == 'click') \
              .groupBy('campaignId') \
              .agg(F.count('action').alias('click_count')) \
              .orderBy(F.desc('click_count'))

# Mostrar o resultado com a campanha com mais cliques no topo
df_clicks.show(1)  # Mostrar apenas a campanha com mais cliques

+-------------+-----------+
|   campaignId|click_count|
+-------------+-----------+
|campaignId_02|      63983|
+-------------+-----------+
only showing top 1 row



**Resposta:**

A campanha com mais clicks foi a campanha de ID 02, com 63.983 cliques.

## 3) Qual mês teve o maior total de eventos acumulado? (2,5 pontos)

In [ ]:
result = data_spark.withColumn('month', F.month('timestamp'))\
   .groupBy('month').agg(F.count('*').alias('event_count')) \
  .orderBy(F.desc('event_count'))

# Mostrar o resultado com o mês com mais eventos
result.show(1)  # Mostrar apenas o mês com mais eventos


+-----+-----------+
|month|event_count|
+-----+-----------+
|    1|      25800|
+-----+-----------+
only showing top 1 row



**Resposta:**

O mês que teve o maior total de eventos acumulados foi janeiro (mês 01), com 25.800 eventos.

## 4) Nas situações onde existe um evento de view seguido de um evento de click criados pelo mesmo usuário no mesmo anúncio e campanha, quais são os 5 pares de anúncio e campanha com menores médias de tempo entre os dois eventos (2,5 pontos)

In [ ]:
view_events = data_spark.filter(data_spark['action'] == 'view')
view_events = view_events.withColumnRenamed("timestamp", "view_timestamp")

In [ ]:
click_events = data_spark.filter(data_spark['action'] == 'click')
click_events = click_events.withColumnRenamed("timestamp", "click_timestamp")

In [ ]:
join_condition = (view_events['user_id'] == click_events['user_id']) & \
                 (view_events['adId'] == click_events['adId']) & \
                 (view_events['campaignId'] == click_events['campaignId']) & \
                 (view_events['view_timestamp'] < click_events['click_timestamp'])

joined_df = view_events.join(click_events, join_condition, 'inner') \
    .select(view_events['user_id'], view_events['adId'], view_events['campaignId'],
            view_events['view_timestamp'], click_events['click_timestamp'])

In [ ]:
joined_df = joined_df.withColumn('time_diff',
                                 (F.unix_timestamp('click_timestamp') - F.unix_timestamp('view_timestamp')))

In [ ]:
avg_time_diff_df = joined_df.groupBy('adId', 'campaignId').avg('time_diff')

In [ ]:
top_5_pairs = avg_time_diff_df.orderBy('avg(time_diff)').limit(5)
top_5_pairs.show()

+-------+-------------+--------------------+
|   adId|   campaignId|      avg(time_diff)|
+-------+-------------+--------------------+
|adId_07|campaignId_01|1.0078583937431393E7|
|adId_02|campaignId_01|1.0154171090081457E7|
|adId_09|campaignId_01| 1.020334868419649E7|
|adId_02|campaignId_03| 1.021572514987986E7|
|adId_02|campaignId_02|1.0226988156816851E7|
+-------+-------------+--------------------+



**Resposta:**


Os 5 pares de anúncio e campanha com menores médias de tempo entre os dois eventos são:


1.   adId_07 e campaignId_01
2.   adId_02 e campaignId_01
3. adId_09 e campaignId_01
4. adId_02 e campaignId_03
5. adId_02 e campaignId_02